In [2]:
from langchain_community.document_loaders import UnstructuredExcelLoader
file_path = "/home/namnh1/rag-llm-chatbot/BC AI/bc.xlsx"
df = UnstructuredExcelLoader(file_path=file_path)
content = df.load()

In [3]:
content

[Document(metadata={'source': '/home/namnh1/rag-llm-chatbot/BC AI/bc.xlsx'}, page_content='TỔNG HỢP CÔNG NỢ OEM TUẦN 2025-03-08 00:00:00 Tuần 1 tháng 3- từ ngày 01/3 đến 7/3/2025 TT Tên đại lý Nợ ĐK Có ĐK PS Nợ Thu tiền Trả lại, CK Nợ CK Có CK KẾ HOẠCH Đặc điểm Ngày PS Đặt hàng Tồn kho Quá hạn tt Ghi chú Ghi chú Ngày đưa xác nhận CN Ngày nhận xác nhận CN HĐNT\\nNỘI BỘ HĐNT\\nTHUẾ CAM KẾT 2024 1 Nguyễn Thị Lan Thu 600000000 170000000 4000000000 341850000 0 3580000000 -8150000 0 1 OEMDUCTOAN\\nPhương: Đức Toàn\\nGold mark 100000000 41850000 -58150000 Bản in 5tr\\nĐặt 30%\\nTT : 1 tháng 2024-04-06 00:00:00 27/6 2024 đã ký 2023,2024 đã ký HB Đã ký 2 OEMHUNGT\\nTrường: Hưng Thịnh\\nMalayone 500000000 Đặt 30%\\nTT ngay 2025-04-03 00:00:00 - 3 OEMNGHIN\\nA Nghinh\\nSunshise 0 70000000 500000000 430000000 Đặt 30%\\nTT ngay 2025-04-03 00:00:00 4/5\\n4/6 13/6 2024 đã ký Không Không 4 OEMHAVITECH\\nHavitech 0 500000000 500000000 Đặt 30%\\nTT ngay 2025-04-03 00:00:00 - 5 OEMKAISE\\nNgọc: Smarttech

In [7]:
import re
import os
import pandas as pd
import datetime
from langchain_ollama import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from docling.document_converter import DocumentConverter

class DocumentProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.chunk_size = 5000
        self.chunk_overlap = 200
        self.embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url="http://localhost:11434")
        self.docs = self.process_documents()

    def preprocess_text(self, text: str) -> str:
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    def preprocess_excel(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Preprocess Excel-specific data:
        - Fill empty cells with a default value.
        - Remove completely empty columns.
        - Normalize data types to strings, handling all datetime types correctly.
        """
        df = df.dropna(axis=1, how='all')  # Remove completely empty columns
        df = df.fillna('')  # Fill empty values with an empty string

        for col in df.columns:
            df[col] = df[col].apply(
                lambda x: (
                    x.strftime('%Y-%m-%d %H:%M:%S') if isinstance(x, (pd.Timestamp, datetime.datetime))
                    else str(x).strip() if isinstance(x, str) and x != '' 
                    else str(x) if x != '' 
                    else ''
                )
            )

        df.columns = [re.sub(r'\s+', '_', col.strip()) for col in df.columns]  # Normalize column names
        return df

    def process_documents(self):
        # converter = DocumentConverter()
        documents = []

        if os.path.isfile(self.file_path):
            file_paths = [self.file_path]
        elif os.path.isdir(self.file_path):
            file_paths = [
                os.path.join(self.file_path, f) for f in os.listdir(self.file_path)
                if f.lower().endswith(('.pdf', '.docx', '.doc', '.xls', '.xlsx'))
            ]
        else:
            raise ValueError(f"Invalid path: {self.file_path}")

        for path in file_paths:
            try:
                # if path.lower().endswith(('.xls', '.xlsx')):
                #     df = pd.read_excel(path)
                #     df = self.preprocess_excel(df)
                #     text_content = df.to_csv(index=False)
                # else:
                #     result = converter.convert(path)
                #     text_content = result.document.export_to_markdown()

                # preprocessed_text = self.preprocess_text(text_content)
                # from langchain.docstore.document import Document
                # doc = Document(page_content=preprocessed_text, metadata={"source": path})
                df = UnstructuredExcelLoader(file_path=path)
                doc = df.load()
                documents.append(doc)

            except Exception as e:
                print(f"Error processing file {path}: {str(e)}")
                continue

        splitter = RecursiveCharacterTextSplitter(chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
        return splitter.split_documents(documents)

directory_path = "/home/namnh1/rag-llm-chatbot/BC AI"
doc_processors = DocumentProcessor(directory_path)

AttributeError: 'list' object has no attribute 'page_content'

In [2]:
from rank_bm25 import BM25Okapi
from sentence_transformers import CrossEncoder
from qdrant_client.http.models import HnswConfigDiff, VectorParams, Distance, PointStruct
from qdrant_client import QdrantClient
from typing import List
import uuid

class QdrantDB:
    def __init__(self, host="10.100.140.54", port=6333, embeddings=None):
        self.client = QdrantClient(host=host, port=port)
        self.embeddings = embeddings
        self.reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
        self.bm25 = None  # Will store the BM25 index after indexing

    def create_collection(self, collection_name):
        existing_collections = [col.name for col in self.client.get_collections().collections]
        if collection_name not in existing_collections:
            self.client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(
                    size=len(self.embeddings.embed_query("wellcome to eBot (DXTech)")),
                    distance=Distance.DOT
                ),
                hnsw_config=HnswConfigDiff(
                    m=16,
                    ef_construct=100,
                    full_scan_threshold=10000
                )
            )
            return True
        return False

    def insert_documents(self, collection_name: str, doc_chunks: List, embeddings):
        vectors = [embeddings.embed_query(chunk.page_content) for chunk in doc_chunks]
        payloads = [{"id": str(uuid.uuid4()), "text": chunk.page_content} for chunk in doc_chunks]
        points = [PointStruct(id=payload["id"], vector=vectors[i], payload=payload) for i, payload in enumerate(payloads)]
        
        self.client.upsert(collection_name=collection_name, points=points)

        # Precompute BM25 index during insertion
        texts = [chunk.page_content for chunk in doc_chunks]
        tokenized_corpus = [text.split() for text in texts]
        self.bm25 = BM25Okapi(tokenized_corpus)

    def rerank_documents(self, question: str, documents: List[str], top_k: int = 3) -> List[str]:
        """
        Rerank a list of documents based on relevance to the question using CrossEncoder.
        """
        if not documents:
            return []
        
        rerank_inputs = [[question, doc] for doc in documents]
        scores = self.reranker.predict(rerank_inputs, batch_size=32)
        sorted_pairs = sorted(zip(scores, documents), reverse=True)
        return [doc for _, doc in sorted_pairs][:top_k]
    
    def search_database_fusion_bm250(self, collection_name, question, embeddings, limit=5):
        if self.bm25 is None:
            # Fetch all documents from Qdrant if BM25 index isn’t precomputed
            all_docs = self.client.scroll(
                collection_name=collection_name,
                limit=10000,  # Adjust based on your collection size
                with_payload=True
            )[0]
            texts = [doc.payload['text'] for doc in all_docs if 'text' in doc.payload]
            tokenized_corpus = [text.split() for text in texts]
            self.bm25 = BM25Okapi(tokenized_corpus)

        # BM25 Search
        bm25_scores = self.bm25.get_scores(question.split())
        top_bm25_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:limit]
        all_docs = self.client.scroll(collection_name=collection_name, limit=10000)[0]  # Fetch all docs
        top_bm25_results = [all_docs[i].payload['text'] for i in top_bm25_indices if i < len(all_docs)]

        # Vector Search
        query_vector = embeddings.embed_query(question)
        search_result = self.client.search(
            collection_name=collection_name,
            query_vector=query_vector,
            limit=limit,
            score_threshold=0.6
        )

        # Fusion Search
        fusion_results = top_bm25_results + [hit.payload['text'] for hit in search_result if 'text' in hit.payload]
        fusion_results = list(set(fusion_results))  # Remove duplicates

        # Rerank combined results
        reranked_docs = self.rerank_documents(question, fusion_results, top_k=limit)
        return "\n\n".join(reranked_docs)

In [3]:
prompt_template = """
Bạn là một trợ lý thông minh, chuyên về phân tích dữ liệu và hỗ trợ truy xuất thông tin (RAG). Nhiệm vụ của bạn là sử dụng dữ liệu và lịch sử hội thoại để đưa ra những phân tích chính xác, tóm tắt thông tin và giải đáp các thắc mắc của người dùng một cách rõ ràng, chi tiết và có căn cứ.
[Context]
{context}

[Conversation History]
{history}

[User Question]
{question}

Your Answer:
"""

In [4]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable.passthrough import RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser

class PipelineBot:
    def __init__(self,qdrant_client, collection_name, embeddings, document_processor, model_name = "deepseek-r1:14b"):
        self.llm = ChatOllama(model=model_name, temperature=0)
        self.qdrant_client = qdrant_client
        self.collection_name = collection_name
        self.embeddings = embeddings
        self.document_processor = document_processor

    
    def build_chain(self, qdrant_client, collection_name, embeddings, question, history):
        # retrieved_context = qdrant_client.search_database_fusion_bm250(collection_name, question, embeddings, self.document_processor.docs)
        retrieved_context = qdrant_client.search_database_fusion_bm250(
            collection_name=collection_name,
            question=question,
            embeddings=embeddings,
            limit=5  # Explicitly pass limit if you want to customize it
        )
        pipeline = (
            RunnablePassthrough()
            | {"history": lambda x: history, "question": lambda x: question, "context": lambda x: retrieved_context}
            | ChatPromptTemplate.from_template(prompt_template)
            | self.llm
            | StrOutputParser()
        )
        result = pipeline.invoke({})
        return result
    def __call__(self, question, history):
        return self.build_chain(self.qdrant_client, self.collection_name, self.embeddings, question, history)

In [5]:
directory_path = "/home/namnh1/rag-llm-chatbot/BC AI"
doc_processors = DocumentProcessor(directory_path)

Lỗi khi xử lý file /home/namnh1/rag-llm-chatbot/BC AI/BC Công nợ OEM tuần.xlsx: 'datetime.datetime' object has no attribute 'strip'


In [21]:
qdrant_db = QdrantDB(embeddings=doc_processors.embeddings) if doc_processors else None

In [22]:
collection_name = "hobiwood"
qdrant_db.create_collection(collection_name)
qdrant_db.insert_documents(collection_name, doc_chunks = doc_processors.docs, embeddings=doc_processors.embeddings)

In [23]:
chatbot = PipelineBot(
        qdrant_client=qdrant_db, 
        collection_name=collection_name, 
        embeddings=doc_processors.embeddings, 
        document_processor=doc_processors
    )

In [38]:
question = "Công ty cổ phần S- Decoro có giá trị hợp đồng là bao nhiêu "
response = chatbot(question, "")

/tmp/ipykernel_1782533/1001733337.py:77: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = self.client.search(


In [39]:
response

'<think>\nĐầu tiên, tôi xem xét dữ liệu được cung cấp để tìm kiếm thông tin về giá trị hợp đồng của Công ty cổ phần S-Decoro. Tuy nhiên, trong dữ liệu này không đề cập đến bất kỳ chi tiết nào liên quan đến công ty này. Tất cả các thông tin đều xoay quanh Công ty Cổ Phần Hải Phòng Invest và một số OEM khác như OEM Ductoan và OEM Hungt.\n\nDo đó, tôi không thể xác định được giá trị hợp đồng của S-Decoro từ dữ liệu hiện tại. Tôi khuyên người dùng nên liên hệ trực tiếp với công ty hoặc tìm kiếm thông tin từ các nguồn chính thức để có được thông tin chính xác.\n</think>\n\nCông ty cổ phần S-Decoro không có thông tin trong dữ liệu được cung cấp.'